# Laboratorio 6 Face Recognition usando OpenCV 

## Introducción

## OpenCV Face Recognizers

OpenCV cuenta con 3 reconocedores faciales integrados. Estos se pueden utilizar de manera independiente en este código simplemente cambiando una línea de código. Estos son: 

1. EigenFaces Face Recognizer Recognizer - `cv2.face.EigenFaceRecognizer_create()`
2. FisherFaces Face Recognizer Recognizer - `cv2.face.FisherFaceRecognizer_create()`
3. Local Binary Patterns Histograms (LBPH) Face Recognizer - `cv2.face.LBPHFaceRecognizer_create()`

En este caso se estará utilizando el reconocedor LBPH. 

### Local Binary Patterns Histograms (LBPH) Face Recognizer 




Una explicación detallada de LBPH puede ser encontrada en [face detection](https://www.superdatascience.com/opencv-face-detection/).


Los reconocedores de Eigen y Fisher son afectados por la luz y esta es una condición que no se puede garantizar en situaciones de la vida real. El reconocedor usando LBPH es una mejora para superar esta desventaja. Su enfoque es utilizar descriptores locales en la imagen. LBPH trata de encontrar una estructura de la imagen y lo hace mediante la comparación de cada pixel con los de su vecindario. 

Se toma una ventana de 3x3 y se mueve a través de la imagen, en cada movimiento se compara el pixel central con los vecinos. Los vecinos con una intensidad menor o igual al del pixel central se marcan utilizando un 1 y los demás con un 0. Estos valores dentro de la ventana se leen en el sentido de las agujas del reloj lo que creará un patrón binario como 11100011 el cual es específico para esta zona de la imagen. Haciendo esto a través de toda la imagen se tendrá una lista de patrones locales binarios. 


**LBP Labeling**

![LBP labeling](img/lbp-labeling.png)

Con lo anterior se tiene la parte de los patrones binarios, para la creación del histograma, se convierte cada patron en un número binario (binario -->  digital) y entonces se realiza un histograma de todos los valores decimales.  

**Sample Histogram**

![LBP labeling](img/histogram.png)

Con este enfoque estaremos creando un histograma para cada cara en la imagen. Por lo cual, cuando tenemos un dataset de entrenamiento con 100 caras tendremos 100 histogramas diferentes que se almacenaran para realizar el proceso de reconocimiento posteriormente. El algoritmo sabe que cara pertenece cada histograma. Durante la etapa de reconocimiento se pasará una imagen al reconocedor, el cual calculará el histograma de la cara detectada en la imagen y lo comparará con los histogramas que tiene almacenados, para devolver la categoría que mejor coincida con la imagen en evaluación. 

En esta imagen podemos ver como los LBPH no son afectados por los factores de iluminación

**LBP Faces**

![LBP faces](img/lbph-faces.jpg)

**[source](http://docs.opencv.org/2.4/modules/contrib/doc/facerec/facerec_tutorial.html)**


# Proceso de Reconocimiento Facial en OpenCV

El proceso de reconocimiento facial se puede dividir en 3 etapas:

1. **Preparar los datos de entrenamiento:** En este paso se leerán las imágenes de entrenamiento para cada persona con sus etiquetas, se detectaran las caras en cada imagen y se asignan a una etiqueta o label entero.
2. **Entrenar el reconocedor:** En este paso entrenaremos el reconocedor de caras de LBPH enviándole/mostrándole la información que se ha preparado en el paso 1. 
3. **Testing:** En esta etapa enviaremos algunas imágenes de prueba, para evaluar si la predicción se realiza de manera correcta. 


In [2]:
#import OpenCV module
import cv2
#import os module for reading training data directories and paths
import os
#import numpy to convert python lists to numpy arrays as 
#it is needed by OpenCV face recognizers
import numpy as np

### Training Data

Entre mayor cantidad de imágenes por sujeto, los resultados serán mejores ya que el reconocedor será capaz de aprender datos de la misma persona desde diferentes puntos de vista. En este caso nuestro dataset tiene 12 imágenes de cada sujeto, los cuales se encuentran en el folder `training-data`, este contiene en su interior un folder para cada sujeto que deseamos reconocer  cada folder tiene el formato `sLabel (e.g. s1, s2)` donde el número es la etiqueta entera asignada a acada sujeto. 


```
training-data
|-------------- s1
|               |-- 1.jpg
|               |-- ...
|               |-- 12.jpg
|-------------- s2
|               |-- 1.jpg
|               |-- ...
|               |-- 12.jpg
```

El folder _`test-data`_ contiene las imágenes que serán utilizadas para evaluar nuestro reconocedor luego de ser entrenado. 

Las etiquetas en OpenCV deben ser de tipo entero, por lo cual se establece una forma de mapeado entre los números y los nombres de las personas. En nuestro caso no se utiliza el 0, por lo cual se deja vacío en la lista que contiene los nombres. 

In [3]:
#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["", "Ruben Blades", "Elvis Presley"]

### Preparar los datos de entrenamiento

Para entrenar el reconocedor OpenCV necesita dos arreglos, uno con los rostros (histograma de patrones) de los sujetos de en el conjunto de entrenamiento y el segundo vector contiene, en el mismo orden, las etiquetas de cada rostro

Por lo cual, si nuestro dataset contiene datos en esta forma:

```
PERSON-1    PERSON-2   

img1        img1         
img2        img2
```

Las listas producidas tendrán la siguiente estructura. 

```
FACES                        LABELS

person1_img1_face              1
person1_img2_face              1
person2_img1_face              2
person2_img2_face              2
```


Esta preparación se puede resumir como: 

1. Procesar la carpeta de entrenamiento, de donde se obtendrán la cantidad de personas que estarán en el reconocedor.
2. Para cada sujeto se debe extraer la etiqueta que se le asignará y almacenarla en formato de entero. 
3. Leer las imágenes para cada personas y detectar la cara en cada una de estas. 
4. Añadir cada cara a la lista de caras y su etiqueta correspondiente a la lista de etiquetas. 

In [4]:
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')

    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

El detector LBP necesita trabajar con imágenes en escala de grises, de igual manera se debe realizar como primer paso un recorte de la ubicación del rostro dentro de la imagen. En este fragmento de código se realiza la conversión de la imagen a escala de grises y se utilizan el `cv2.CascadeClassifier` mediante un modelo de detección de frontal de rostros, para realizar el recorte de la cara en la imagen. Este método devuelve el (x, y, width, height) de la zona en la cual se encuentra el rostro, para que pueda ser extraida utilizando OpenCV


In [5]:
#this function will read all persons' training images, detect face from each image
#and will return two lists of exactly same size, one list 
# of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        
        #build path of directory containin images for current subject subject
        #sample path_dir_subject = "training-data/s1"
        path_dir_subject = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(path_dir_subject)
        
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = path_dir_subject + "/" + image_name

            #read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detect face
            face, rect = detect_face(image)
            
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1) 
    cv2.destroyAllWindows()
    
    return faces, labels

Esta función cumple la misión de preparar los datos de entrenamiento, recibiendo la ruta de la carpeta de entrenamiento y devolviendo las listas de caras y etiquetas de cada cara.

In [6]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  22
Total labels:  22


### Train Face Recognizer

In [7]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#or use EigenFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.EigenFaceRecognizer_create()

#or use FisherFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.FisherFaceRecognizer_create()

En este paso procedemos a instanciar el reconocedor y posteriormente se realiza el entrenamiento del mismo utilizando el método  `train(faces-vector, labels-vector)` el cual recibe la lista de caras y etiquetas del conjunto de entrenamiento. 

In [8]:
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

### Predicción

In [9]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

Estas funciones dibujarán un rectángulo sobre el rostro detectado y escribiran la etiqueta que ha definido el detector que corresponde con el rostro. 

In [10]:
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label= face_recognizer.predict(face)
    print(label[1])  #valor de confidence, es una distancia entre más pequeño más cerca por lo tanto mejor
    #get name of respective label returned by face recognizer
    label_text = subjects[label[0]]
    
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img

En este caso utilizamos el reconocedor entrenado para definir una etiqueta para un rostro en una imagen de prueba. Para esto se utiliza el método `predict(face)`, este retorna una tupla que contendrá el label(entero) al cual el reconocedor asignó la imagen y también un valor de confianza/probabilidad de dicho resultado. 



In [11]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("test-data/test0.jpg")
test_img2 = cv2.imread("test-data/test6.jpg")

###Si no detecta caras en la imagen dará un error

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.imshow(subjects[2], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
60.978268227083475
26.689962927284107
Prediction complete
